In [ ]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.2f}'.format    # pandas: para todos os números aparecerem com duas casas decimais

Para salvar os dados:
- fui em https://sidra.ibge.gov.br/tabela/6957
- selecionei quantidade produzida e valor da produção total de cana, milho, soja, sorgo e trigo (dados de 2017)
- em todas as opções eu escolhi Total
- 'unidade territorial': municípios
- fiz o download da planilha como xlsx
- o nome do arquivo ficou 'tabela6957.xlsx'

# Quantidade produzida (toneladas) em 2017

Antes, carregar os dados no ambiente colab (na pastinha à esquerda, fazer upload do arquivo tabela6957.xlsx).

In [ ]:
# sheet_name=0 significa a primeira planilha do arquivo
df = pd.read_excel('tabela6957.xlsx', sheet_name=0)

Vamos ver as primeiras linhas do *dataframe*:

In [ ]:
df.head()

Vamos realizar algumas alterações:

- começar a ler da linha 6, ou seja, vamos pular 5 linhas
- a última linha também é desnecessária, vamos desconsiderá-la
- não queremos ler as colunas 0, 3 e 4 (Nível, Condições, Grupos de atividade econômica)
- para ajudar a visualizar os dados, vamos já alterar os nomes das colunas

In [ ]:
# criar uma lista com os índices das colunas a retirar
pular = [0, 3, 4]
# criar uma lista com os índices das colunas a manter
cols = [i for i in range(len(df.columns)) if i not in pular]
# índices das colunas a manter
cols

In [ ]:
# ler os dados desconsiderando as 5 primeiras linhas e a última linha
# considerar as colunas que estão na lista 'cols'
df = pd.read_excel('tabela6957.xlsx', skiprows=5, skipfooter=1, usecols=cols, sheet_name=0)
# alterar os nomes das colunas
df.columns = ['mun', 'nome_mun', 'cana', 'milho', 'soja', 'sorgo', 'trigo']

In [ ]:
df.head()

Número de observações e variáveis do conjunto de dados:

In [ ]:
df.shape

- A coluna 'município' tem o nome do município e, entre parênteses, a sigla do estado.
- Vamos usar a função str.split para dividir o conteúdo a partir de '(':

In [ ]:
# a opção 'expand=True' divide a string e já permite criar as colunas com os
# conteúdos divididos
df[['nome_mun', 'uf']] = df['nome_mun'].str.split('(', expand=True)
df.head()

- A coluna 'sigla_uf' tem o ')' no final.
- Vamos usar a função str.replace para substituir ')' por nada
- regex significa 'expressões regulares', uma sequência de caracteres que podem ser pesquisados, substituídos etc.

In [ ]:
df['uf'] = df['uf'].str.replace(')', '', regex=True)
df.head()

In [ ]:
df.columns

In [ ]:
# reorganizar as colunas
df = df.loc[:, ['mun', 'nome_mun', 'uf', 'cana', 'milho', 'soja', 'sorgo', 'trigo']]

Há valores faltantes?

In [ ]:
df.isna().sum()

In [ ]:
print(df[['cana']].value_counts())
print(df[['milho']].value_counts())
print(df[['soja']].value_counts())
print(df[['sorgo']].value_counts())
print(df[['trigo']].value_counts())

Nas notas da planilha, há as seguintes informações:

* -: zero absoluto, não resultante de um cálculo ou arredondamento.
* 0: zero resultante de um cálculo ou arredondamento.
* X: valor inibido para não identificar o informante.
* ..: valor não se aplica
* ...: valor não disponível


Primeira abordagem: substituir todos esses valores por 0.

Mas antes, vamos fazer uma cópia dos dados que já trabalhamos:

In [ ]:
df1 = df.copy()

In [ ]:
df1 = df1.replace(['-', '...', 'X'], 0)

Antes de limpar mais os dados, vamos visualizar o resumo estatístico:

In [ ]:
# resumo estatístico
df1.iloc[:, 3:].describe()

Agora vamos adotar outra estratégia:
- substituir por 0 se aparecer -
- retirar as linhas em que aparecerem X, .. ou ...

Vamos alterar o próprio *dataframe* df:

In [ ]:
df = df.replace(['-'], 0)
df = df.replace(['X', '..', '...'], np.nan).dropna(axis=0, how='any')
df.head()

In [ ]:
df.iloc[:, 3:].describe()

In [ ]:
# checar os tipos das variáveis
df.info()

In [ ]:
df.shape

Se quisermos salvar o novo arquivo:

In [ ]:
df.to_excel('qtde-6957.xlsx', index=False)

## Filtrar dados

E se quisermos selecionar uma parte dos dados: um estado, uma região etc.?


In [ ]:
df.sample(5)

In [ ]:
# podemos selecionar um ou mais estados, por exemplo
dados = df.query('uf == "MG"')

In [ ]:
# começar os índices do zero
dados = dados.reset_index(drop=True)

In [ ]:
dados.head()

In [ ]:
dados.shape

In [ ]:
dados.to_excel('qtde-mg.xlsx', index=False)

In [ ]:
# podemos selecionar um ou mais estados, por exemplo
dados = df.query('(uf == "MG") or (uf == "SP")')
dados = dados.reset_index(drop=True)   # zerar os índices
dados.shape

In [ ]:
dados.sample(10)

## Unir os dados com outras informações sobre os municípios

Dessa forma podemos incorporar novas informações ao *dataframe*.

In [ ]:
df.sample(5)

In [ ]:
# dados do Brasil todo
df.shape

In [ ]:
df.info()

Se alguma variável que precisa ser de um tipo e não for:

In [ ]:
# df['mun'] = df['mun'].astype(int)  # transformar códigos em inteiros caso não forem

Podemos unir esses dados com um outro que possui outras informações, como mesorregiões, microrregiões etc. e realizar outras filtragens:

In [ ]:
# informações sobre as observações
cod = pd.read_csv('https://raw.githubusercontent.com/patriciasiqueira/patriciasiqueira.github.io/master/arquivos/codigos-municipios.csv', encoding='utf8')
cod.head()

Se precisarmos mudar o nome de alguma coluna de um dos *dataframes* para coincidir:

In [ ]:
# cod.rename(columns={'codmun6': 'cod'}, inplace=True)  # mudar nome da coluna 'codmun6' para 'cod', por exemplo

Vamos selecionar algumas variáveis de 'cod' para utilizar:

In [ ]:
cod = cod.loc[:, ['mun', 'nome_micro', 'nome_meso', 'nome_regiao']]

In [ ]:
# mesclar os dois dataframes
df3 = df.merge(cod, on='mun')
df3.head()

Se quisermos tirar alguma variável (coluna):

In [ ]:
# apagar coluna redundante com o nome do município
df3 = df3.drop('nome_micro', axis=1)

In [ ]:
df3.head()

In [ ]:
df3.to_excel('producao-qtde-br.xlsx', index=False)

# Valor da produção (milhares de reais) em 2017

Antes, carregar os dados no ambiente colab (na pastinha à esquerda, fazer upload do arquivo tabela6957.xlsx).

In [ ]:
# sheet_name=1 significa a segunda planilha do arquivo
df = pd.read_excel('tabela6957.xlsx', sheet_name=1)

Vamos ver as primeiras linhas do *dataframe*:

In [ ]:
df.head()

Vamos realizar algumas alterações:

- começar a ler da linha 6, ou seja, vamos pular 5 linhas
- a última linha também é desnecessária, vamos desconsiderá-la
- não queremos ler as colunas 0, 3 e 4 (Nível, Condições, Grupos de atividade econômica)
- para ajudar a visualizar os dados, vamos já alterar os nomes das colunas

In [ ]:
# criar uma lista com os índices das colunas a retirar
pular = [0, 3, 4]
# criar uma lista com os índices das colunas a manter
cols = [i for i in range(len(df.columns)) if i not in pular]
# índices das colunas a manter
cols

In [ ]:
# ler os dados desconsiderando as 5 primeiras linhas e a última linha
# considerar as colunas que estão na lista 'cols'
df = pd.read_excel('tabela6957.xlsx', skiprows=5, skipfooter=1, usecols=cols, sheet_name=1)
# alterar os nomes das colunas
df.columns = ['mun', 'nome_mun', 'cana', 'milho', 'soja', 'sorgo', 'trigo']

In [ ]:
df.head()

Número de observações e variáveis do conjunto de dados:

In [ ]:
df.shape

- A coluna 'município' tem o nome do município e, entre parênteses, a sigla do estado.
- Vamos usar a função str.split para dividir o conteúdo a partir de '(':

In [ ]:
# a opção 'expand=True' divide a string e já permite criar as colunas com os
# conteúdos divididos
df[['nome_mun', 'sigla_uf']] = df['nome_mun'].str.split('(', expand=True)
df.head()

- A coluna 'sigla_uf' tem o ')' no final.
- Vamos usar a função str.replace para substituir ')' por nada
- regex significa 'expressões regulares', uma sequência de caracteres que podem ser pesquisados, substituídos etc.

In [ ]:
df['sigla_uf'] = df['sigla_uf'].str.replace(')', '', regex=True)
df.head()

In [ ]:
df.columns

In [ ]:
# reorganizar as colunas
df = df.loc[:, ['mun', 'nome_mun', 'sigla_uf', 'cana', 'milho', 'soja', 'sorgo', 'trigo']]

Há valores faltantes?

In [ ]:
df.isna().sum()

In [ ]:
print(df[['cana']].value_counts())
print(df[['milho']].value_counts())
print(df[['soja']].value_counts())
print(df[['sorgo']].value_counts())
print(df[['trigo']].value_counts())

Nas notas da planilha, há as seguintes informações:

* -: zero absoluto, não resultante de um cálculo ou arredondamento.
* 0: zero resultante de um cálculo ou arredondamento.
* X: valor inibido para não identificar o informante.
* ..: valor não se aplica
* ...: valor não disponível


Primeira abordagem: substituir todos esses valores por 0.

Mas antes, vamos fazer uma cópia dos dados que já trabalhamos:

In [ ]:
df1 = df.copy()

In [ ]:
df1 = df1.replace(['-', '...', 'X'], 0)

Antes de limpar mais os dados, vamos visualizar o resumo estatístico:

In [ ]:
# resumo estatístico
df1.iloc[:, 3:].describe()

Agora vamos adotar outra estratégia:
- substituir por 0 se aparecer -
- retirar as linhas em que aparecerem X, .. ou ...

Vamos fazer outra cópia do *dataframe* df:

In [ ]:
df2 = df.copy()

In [ ]:
df2 = df2.replace(['-'], 0)
df2 = df2.replace(['X', '..', '...'], np.nan).dropna(axis=0, how='any')
df2.head()

In [ ]:
df2.iloc[:, 3:].describe()

Se quisermos salvar o novo arquivo:

In [ ]:
df2.shape

In [ ]:
df2.to_excel('valor-6957.xlsx', index=False)